<a href="https://colab.research.google.com/github/seirah-yang/Colabwork/blob/main/MLP_Progression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [18]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [19]:
#데이터 불러오기
df = pd.read_csv('/content/drive/MyDrive/wine.csv')
df.head(2)

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality,class
0,7.4,0.70,0.0,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5,1
1,7.8,0.88,0.0,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5,1


In [20]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6497 entries, 0 to 6496
Data columns (total 13 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   fixed acidity         6497 non-null   float64
 1   volatile acidity      6497 non-null   float64
 2   citric acid           6497 non-null   float64
 3   residual sugar        6497 non-null   float64
 4   chlorides             6497 non-null   float64
 5   free sulfur dioxide   6497 non-null   float64
 6   total sulfur dioxide  6497 non-null   float64
 7   density               6497 non-null   float64
 8   pH                    6497 non-null   float64
 9   sulphates             6497 non-null   float64
 10  alcohol               6497 non-null   float64
 11  quality               6497 non-null   int64  
 12  class                 6497 non-null   int64  
dtypes: float64(11), int64(2)
memory usage: 660.0 KB


In [21]:
df.columns

Index(['fixed acidity', 'volatile acidity', 'citric acid', 'residual sugar',
       'chlorides', 'free sulfur dioxide', 'total sulfur dioxide', 'density',
       'pH', 'sulphates', 'alcohol', 'quality', 'class'],
      dtype='object')

In [22]:
#x,y정의
X = df[['fixed acidity', 'volatile acidity', 'citric acid', 'residual sugar',
       'chlorides', 'free sulfur dioxide', 'total sulfur dioxide', 'density',
       'pH', 'sulphates', 'quality', 'class']]

y = df ['alcohol']

In [27]:
#x,y 8:2 쪼개기
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2)

In [28]:
X_train.shape, X_valid.shape, y_train.shape, y_valid.shape

((5197, 12), (1300, 12), (5197,), (1300,))

In [29]:
#배치화
import torch
from torch.utils.data import TensorDataset
from torchvision.transforms import ToTensor

        #파이토치에서는 텐서타입을 쓰니까
X_train = torch.FloatTensor(X_train.values)
X_valid = torch.FloatTensor(X_valid.values)
y_train =  torch.FloatTensor(y_train.values)
y_valid =  torch.FloatTensor(y_valid.values)
        #x,y를 짝꿍을 만들어서 배치로 자른다.
train_set = TensorDataset(X_train, y_train)
valid_set = TensorDataset(X_valid, y_valid)

train_loader = DataLoader(train_set, batch_size=8, shuffle = True)
valid_loader = DataLoader(valid_set, batch_size=8, shuffle = False)


In [38]:
#모델 정의
import torch.nn.functional as F

class MLPProgression(nn.Module):
    def __init__(self):
        super().__init__()
        self.layer1 = nn.Linear(12, 4)
        self.layer2 = nn.Linear(4, 1)

    def forward(self,  x):
        x = F.relu(self.layer1(x))
        x = self.layer2(x)
        return x

model = MLPProgression()


In [39]:
#test : Train loader에서 첫번째 배치 꺼낸다음 0번째 부분인 x만

model(next(iter(train_loader))[0])

tensor([[9.0111],
        [7.5885],
        [5.3131],
        [1.4489],
        [5.5829],
        [4.2137],
        [4.7276],
        [6.2427]], grad_fn=<AddmmBackward0>)

In [45]:
#optimizer, loss 정의

criterion = nn.MSELoss()
optimizer= optim.Adam(model.parameters(), lr=0.001)

In [54]:
#학습
epochs=10

for i in range(epochs):
    for idx, data in enumerate(train_loader):
        X,y = data
        pred = model(X)
        loss = criterion(pred, y)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print(f"epoch: {i+1} loss{loss}")


/usr/local/lib/python3.12/dist-packages/torch/nn/modules/loss.py:616: UserWarning: Using a target size (torch.Size([5])) that is different to the input size (torch.Size([5, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


epoch: 1 loss3.1541192531585693
epoch: 2 loss1.6083542108535767
epoch: 3 loss0.7409921884536743
epoch: 4 loss1.840370535850525
epoch: 5 loss0.374327689409256
epoch: 6 loss0.5853731632232666
epoch: 7 loss1.9457441568374634
epoch: 8 loss2.232759714126587
epoch: 9 loss2.5081288814544678
epoch: 10 loss2.1270744800567627


In [56]:
#평가

temp = []
for idx, data in enumerate(valid_loader):
    X,y = data
    pred = model(X)
    loss = criterion(pred,y)
    temp.append(loss)

sum(temp)/len(temp)

/usr/local/lib/python3.12/dist-packages/torch/nn/modules/loss.py:616: UserWarning: Using a target size (torch.Size([4])) that is different to the input size (torch.Size([4, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


tensor(1.5721, grad_fn=<DivBackward0>)